# 102 Dismonstrating the classification performance for MolMM
* including comparison and ablation study

In [1]:
import pandas as pd
import numpy as np
import torch
import os
import itertools
from collections import OrderedDict, defaultdict

In [2]:
def show_result(path,begin,count,stage,start='test'):
    auc_dict = defaultdict(list)
    for num in range(begin,begin+count+1,1):
        name= path+'/version_'+str(num)+'/metrics.csv'
        df = pd.read_csv(name)
        col = np.array(df.columns)
        mask_col = [ s.startswith(start) and ('total' not in s) and (('auc' in s) or ('acc' in s) or ('loss' in s)) for s in col ]
        df = df[col[mask_col]]
        for s in df.columns:
            if (stage=='test'):
                auc_dict[s].extend(df[s].to_numpy()[0::2])
            elif (stage=='fda_test'):
                auc_dict[s].extend(df[s].to_numpy()[1::2])
                
    name = []
    loss=defaultdict(list)
    auc = defaultdict(list)
    acc = defaultdict(list)
    for key in auc_dict.keys():   
        tmp = np.array(auc_dict[key])
        
        if 'acc' in key:
            acc[key].extend(tmp)
        if 'auc' in key and  'pr' not in key:
            name.append(key[5:-4])
            auc[key].extend(tmp)
        if 'pr' in key:
            # print(key)
            loss[key].extend(tmp)
        
    return loss,acc,auc,name

def print_sorted_results(dicts,startw=None,starte=None):
    result_dict=defaultdict(list)
    for k,v in dicts.items():
        task_s=k.split('_')
        title='_'.join([startw,task_s[1],starte])
        result_dict[title].extend(v)
    return result_dict

def show(key,nkey=None,loss=False,acc=False):
    import os
    import re
    from collections import defaultdict
    global df_total

    folder_path = 'results'

    ckpt_dict = defaultdict(str)
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            if ('metrics' in filename):
                pathlist=os.path.join(dirpath, filename).split('/')
                ckpt_dict[pathlist[1]+'/'+pathlist[2]] = os.path.join(dirpath, filename)
                
    num_dict = defaultdict(int)
    for k,v in ckpt_dict.items():
        num = int(re.findall(r"version_(.+?)/",v)[-1])
        num_dict[k] = num
        
    # print(''.join(['-' for _ in range(50)]))
    # print()
    # # print('_'.join(key))
    # if not (np.array([ (np.array([ keys in k for keys in key]).all() and \
    #         (nkey is None or np.array([ keys not in k for keys in nkey]).all())) for k,v in num_dict.items() ]).any() ):
    #     print('None')
    
    merge_result_dict=defaultdict(list)
    for k,v in num_dict.items():
        result_dict=defaultdict(list)
        if (np.array([ keys in k for keys in key]).all() and \
            (nkey is None or np.array([ keys not in k for keys in nkey]).all())):
            
            loss1,acc1,auc1,name1 = show_result('./results/'+k,0,num_dict[k],"test")
            loss2,acc2,auc2,name2 = show_result('./results/'+k,0,num_dict[k],"fda_test")
            
            f=lambda x: np.array(list(x.values()))[0]
            f1=lambda x,y,s: {'_'.join([list(x.keys())[0],s]):(f(x)+f(y))/2}
                 
            loss_m=f1(loss1,loss2,'pr')
            acc_m=f1(acc1,acc2,'acc')
            auc_m=f1(auc1,auc2,'auc')
            
            nk='_'.join(k.split('/')[1].split('_')[:1]+k.split('/')[1].split('_')[3:])
            sw_test='_'.join([nk,'N'])
            sw_refine_test='_'.join([nk,'R'])
            sw_mean='_'.join([nk,'M'])
            
            # print(k,'K_FOLD_NUM',num_dict[k])
            
            if loss:
                # print(loss1)
                result_dict={**result_dict,**print_sorted_results(loss1,sw_test,'pr')}
                result_dict={**result_dict,**print_sorted_results(loss2,sw_refine_test,'pr')}
                result_dict={**result_dict,**print_sorted_results(loss_m,sw_mean,'pr')}

            if acc:
                result_dict={**result_dict,**print_sorted_results(acc1,sw_test,'acc')}
                result_dict={**result_dict,**print_sorted_results(acc2,sw_refine_test,'acc')}
                result_dict={**result_dict,**print_sorted_results(acc_m,sw_mean,'acc')}
                
            result_dict={**result_dict,**print_sorted_results(auc1,sw_test,'auc')}
            result_dict={**result_dict,**print_sorted_results(auc2,sw_refine_test,'auc')}
            result_dict={**result_dict,**print_sorted_results(auc_m,sw_mean,'auc')}
            
            # for k,v in result_dict.items():
            #     print(k,v)
            # print()
        merge_result_dict={**merge_result_dict,**result_dict}
    
    df_total={**df_total,**merge_result_dict}
    
    # return merge_result_dict
    # print()

In [3]:
sort_results_dict = lambda dicts: sorted(dicts.items(), key=lambda v: v[1][0], reverse=True)
list_show = lambda x: None if [ print(k,v) for (k,v) in x ] else None
list_origin = lambda dicts: None if [ print(k,v) for k,v in dicts.items() ] else None
def show_df(task,dataset,metric,sort=True):
    global df_total
    df_total_mean={k:(round(np.array(v).mean(),4),round(np.array(v).std(),4)) for k,v in df_total.items()}
    task_metric={ k:v for k,v in df_total_mean.items() if task in k and dataset in k.split('_') and metric in k}
    # task_acc={ k:v for k,v in df_total_mean.items() if task in k and dataset in k and 'acc' in k}
    print(task,dataset,metric)
    if sort:
        list_show(sort_results_dict(task_metric))
    else:
        list_origin(task_metric)
    print()

In [4]:
df_total=defaultdict(list)
print('Comparison study: baseline models vs. MolMM')
# show(['baseline_sub','DNN'],[],acc=True,loss=True)
# show(['baseline_inh','DNN'],[],acc=True,loss=True)
# show(['baseline_sub','pure'],['DNN','stop'],acc=True,loss=True)
# show(['baseline_inh','pure'],['DNN','stop'],acc=True,loss=True)
show(['baseline_sub'],['merge','pure','stop'],acc=True,loss=True)
show(['baseline_inh'],['merge','pure','stop'],acc=True,loss=True)
show(['multitask_sub'],['stop'],acc=True,loss=True)
show(['multitask_inh'],['stop'],acc=True,loss=True)
show(['meta_sub','meta'],['stop','guide'],acc=True,loss=True)
show(['meta_inh','meta'],['stop','guide'],acc=True,loss=True)
show(['meta_sub','meta','guide','inv'],['DNN','ab','stop'],acc=True,loss=True)
show(['meta_inh','meta','guide','inv'],['DNN','ab','stop'],acc=True,loss=True)
print()

print('SUB')
# show_df('sub','N','auc')
# show_df('sub','N','acc')
show_df('sub','R','auc',)
show_df('sub','R','acc')
print('INH')
# show_df('inh','N','auc')
# show_df('inh','N','acc')
show_df('inh','R','auc')
show_df('inh','R','acc')

Comparison study: baseline models vs. MolMM

SUB
sub R auc
CNN_val_meta_guide_inv_freeze_R_substrates_auc (0.8126, 0.0313)
CNN_val_meta_guide_inv_full_R_substrates_auc (0.8097, 0.0454)
CNN_val_meta_full_R_substrates_auc (0.8037, 0.0223)
CNN_val_multi_full_R_substrates_auc (0.7987, 0.0334)
CNN_val_multi_freeze_R_substrates_auc (0.7887, 0.0367)
CNN_val_full_R_substrates_auc (0.7647, 0.0156)
CNN_val_freeze_R_substrates_auc (0.664, 0.0293)
CNN_val_meta_freeze_R_substrates_auc (0.5595, 0.1055)

sub R acc
CNN_val_meta_guide_inv_freeze_R_substrates_acc (0.7467, 0.0246)
CNN_val_meta_full_R_substrates_acc (0.7381, 0.0435)
CNN_val_meta_guide_inv_full_R_substrates_acc (0.7338, 0.0564)
CNN_val_multi_full_R_substrates_acc (0.731, 0.0315)
CNN_val_multi_freeze_R_substrates_acc (0.7181, 0.0499)
CNN_val_full_R_substrates_acc (0.7053, 0.0177)
CNN_val_freeze_R_substrates_acc (0.6323, 0.0363)
CNN_val_meta_freeze_R_substrates_acc (0.5894, 0.026)

INH
inh R auc
CNN_nval_meta_guide_inv_freeze_R_inhibitors_au

In [5]:
df_total=defaultdict(list)
print('Ablation study: feature extractor component')
show(['baseline_sub','pure'],['DNN'],acc=True,loss=True)
show(['baseline_inh','pure'],['DNN'],acc=True,loss=True)
show(['baseline_sub','pure','DNN'],[],acc=True,loss=True)
show(['baseline_inh','pure','DNN'],[],acc=True,loss=True)
show(['meta_sub','guide','inv'],['DNN','ab','stop'],acc=True,loss=True)
show(['meta_inh','guide','inv'],['DNN','ab','stop'],acc=True,loss=True)
show(['meta_sub','guide','inv','DNN'],['ab','stop'],acc=True,loss=True)
show(['meta_inh','guide','inv','DNN'],['ab','stop'],acc=True,loss=True)
print()
print('SUB')
# show_df('sub','N','auc')
# show_df('sub','N','acc')
show_df('sub','R','auc')
show_df('sub','R','acc')
print('INH')
# show_df('inh','N','auc')
# show_df('inh','N','acc')
show_df('inh','R','auc')
show_df('inh','R','acc')

Ablation study: feature extractor component

SUB
sub R auc
CNN_val_pure_R_substrates_auc (0.8183, 0.0215)
CNN_val_meta_guide_inv_freeze_R_substrates_auc (0.8126, 0.0313)
CNN_val_meta_guide_inv_full_R_substrates_auc (0.8097, 0.0454)
DNN_val_pure_R_substrates_auc (0.7547, 0.0229)
DNN_val_meta_guide_inv_full_R_substrates_auc (0.7491, 0.0286)
DNN_val_meta_guide_inv_freeze_R_substrates_auc (0.7341, 0.0192)

sub R acc
CNN_val_pure_R_substrates_acc (0.7467, 0.0445)
CNN_val_meta_guide_inv_freeze_R_substrates_acc (0.7467, 0.0246)
CNN_val_meta_guide_inv_full_R_substrates_acc (0.7338, 0.0564)
DNN_val_meta_guide_inv_full_R_substrates_acc (0.7324, 0.0321)
DNN_val_pure_R_substrates_acc (0.7167, 0.0281)
DNN_val_meta_guide_inv_freeze_R_substrates_acc (0.7052, 0.0269)

INH
inh R auc
DNN_nval_meta_guide_inv_freeze_R_inhibitors_auc (0.8349, 0.0291)
CNN_nval_meta_guide_inv_freeze_R_inhibitors_auc (0.8333, 0.0501)
DNN_nval_meta_guide_inv_full_R_inhibitors_auc (0.8207, 0.0387)
CNN_nval_meta_guide_inv_full_R

In [6]:
df_total=defaultdict(list)
print('Ablation study: MAML component')
# show(['baseline_sub','pure'],['DNN'],acc=True,loss=True)
# show(['baseline_inh','pure'],['DNN'],acc=True,loss=True)
show(['multitask_sub'],['stop'],acc=True,loss=True)
show(['multitask_inh'],['stop'],acc=True,loss=True)
show(['meta_sub','meta','guide'],['inv','ab','stop'],acc=True,loss=True)
show(['meta_inh','meta','guide'],['inv','ab','stop'],acc=True,loss=True)
# show(['meta_sub','guide','inv'],['DNN','ab','stop'],acc=True,loss=True)
# show(['meta_inh','guide','inv'],['DNN','ab','stop'],acc=True,loss=True)
print()
print('SUB')
# show_df('sub','N','auc')
# show_df('sub','N','acc')
show_df('sub','R','auc')
show_df('sub','R','acc')
print('INH')
# show_df('inh','N','auc')
# show_df('inh','N','acc')
show_df('inh','R','auc')
show_df('inh','R','acc')

Ablation study: MAML component

SUB
sub R auc
CNN_val_multi_full_R_substrates_auc (0.7987, 0.0334)
CNN_val_multi_freeze_R_substrates_auc (0.7887, 0.0367)
CNN_val_meta_guide_full_R_substrates_auc (0.7882, 0.0291)
CNN_val_meta_guide_freeze_R_substrates_auc (0.7265, 0.0427)

sub R acc
CNN_val_multi_full_R_substrates_acc (0.731, 0.0315)
CNN_val_meta_guide_full_R_substrates_acc (0.7196, 0.0205)
CNN_val_multi_freeze_R_substrates_acc (0.7181, 0.0499)
CNN_val_meta_guide_freeze_R_substrates_acc (0.6823, 0.0356)

INH
inh R auc
CNN_nval_multi_full_R_inhibitors_auc (0.7914, 0.0773)
CNN_nval_multi_freeze_R_inhibitors_auc (0.7897, 0.1026)
CNN_nval_meta_guide_full_R_inhibitors_auc (0.7761, 0.0389)
CNN_nval_meta_guide_freeze_R_inhibitors_auc (0.6847, 0.1463)

inh R acc
CNN_nval_multi_freeze_R_inhibitors_acc (0.7141, 0.0585)
CNN_nval_meta_guide_full_R_inhibitors_acc (0.7027, 0.0351)
CNN_nval_multi_full_R_inhibitors_acc (0.6795, 0.0913)
CNN_nval_meta_guide_freeze_R_inhibitors_acc (0.6316, 0.1088)



In [7]:
df_total=defaultdict(list)
print('Ablation study: MTL components')
# show(['baseline_sub','pure'],['DNN'],acc=True,loss=True)
# show(['baseline_inh','pure'],['DNN'],acc=True,loss=True)
show(['meta_sub','meta'],['inv','guide','ab','stop','freeze'],acc=True)
show(['meta_inh','meta'],['inv','guide','ab','stop','freeze'],acc=True)
show(['meta_sub','meta','guide','inv','ab'],['DNN','stop','freeze'],acc=True,loss=True)
show(['meta_inh','meta','guide','inv','ab'],['DNN','stop','freeze'],acc=True,loss=True)
show(['meta_sub','meta','guide','inv'],['DNN','ab','stop'],acc=True,loss=True)
show(['meta_inh','meta','guide','inv'],['DNN','ab','stop'],acc=True,loss=True)
print()
print('SUB')
# show_df('sub','N','auc')
# show_df('sub','N','acc')
show_df('sub','R','auc')
show_df('sub','R','acc')
print('INH')
# show_df('inh','N','auc')
# show_df('inh','N','acc')
show_df('inh','R','auc')
show_df('inh','R','acc')

Ablation study: MTL components

SUB
sub R auc
CNN_val_meta_guide_inv_freeze_R_substrates_auc (0.8126, 0.0313)
CNN_val_meta_guide_inv_full_R_substrates_auc (0.8097, 0.0454)
CNN_val_meta_guide_inv_ab_full_R_substrates_auc (0.8047, 0.0163)
CNN_val_meta_full_R_substrates_auc (0.8037, 0.0223)

sub R acc
CNN_val_meta_guide_inv_freeze_R_substrates_acc (0.7467, 0.0246)
CNN_val_meta_guide_inv_ab_full_R_substrates_acc (0.7453, 0.0248)
CNN_val_meta_full_R_substrates_acc (0.7381, 0.0435)
CNN_val_meta_guide_inv_full_R_substrates_acc (0.7338, 0.0564)

INH
inh R auc
CNN_nval_meta_guide_inv_freeze_R_inhibitors_auc (0.8333, 0.0501)
CNN_nval_meta_guide_inv_full_R_inhibitors_auc (0.8142, 0.0362)
CNN_nval_meta_guide_inv_ab_full_R_inhibitors_auc (0.7889, 0.032)
CNN_nval_meta_full_R_inhibitors_auc (0.7845, 0.0563)

inh R acc
CNN_nval_meta_guide_inv_full_R_inhibitors_acc (0.7627, 0.0701)
CNN_nval_meta_guide_inv_freeze_R_inhibitors_acc (0.7385, 0.0521)
CNN_nval_meta_full_R_inhibitors_acc (0.7025, 0.0489)
CNN_

In [8]:
df_total=defaultdict(list)
print('Ablation study: multi-fidelity setting')
# show(['baseline_sub','pure'],['DNN'],acc=True,loss=True)
# show(['baseline_inh','pure'],['DNN'],acc=True,loss=True)
show(['baseline_sub','merge'],['stop'],acc=True,loss=True)
show(['baseline_inh','merge'],['stop'],acc=True,loss=True)
show(['multitask_sub'],['stop'],acc=True,loss=True)
show(['multitask_inh'],['stop'],acc=True,loss=True)
show(['meta_sub','meta','guide','inv'],['DNN','ab','stop'],acc=True,loss=True)
show(['meta_inh','meta','guide','inv'],['DNN','ab','stop'],acc=True,loss=True)
show(['meta_sub','meta','guide'],['DNN','inv','ab','stop'],acc=True,loss=True)
show(['meta_inh','meta','guide'],['DNN','inv','ab','stop'],acc=True,loss=True)
print()
print('SUB')
# show_df('sub','N','auc')
# show_df('sub','N','acc')
show_df('sub','R','auc')
show_df('sub','R','acc')
print('INH')
# show_df('inh','N','auc')
# show_df('inh','N','acc')
show_df('inh','R','auc')
show_df('inh','R','acc')

Ablation study: multi-fidelity setting

SUB
sub R auc
CNN_val_meta_guide_inv_freeze_R_substrates_auc (0.8126, 0.0313)
CNN_val_meta_guide_inv_full_R_substrates_auc (0.8097, 0.0454)
CNN_val_multi_full_R_substrates_auc (0.7987, 0.0334)
CNN_val_multi_freeze_R_substrates_auc (0.7887, 0.0367)
CNN_val_meta_guide_full_R_substrates_auc (0.7882, 0.0291)
CNN_val_merge_freeze_R_substrates_auc (0.7692, 0.0347)
CNN_val_merge_full_R_substrates_auc (0.7642, 0.0355)
CNN_val_meta_guide_freeze_R_substrates_auc (0.7265, 0.0427)

sub R acc
CNN_val_meta_guide_inv_freeze_R_substrates_acc (0.7467, 0.0246)
CNN_val_meta_guide_inv_full_R_substrates_acc (0.7338, 0.0564)
CNN_val_multi_full_R_substrates_acc (0.731, 0.0315)
CNN_val_merge_freeze_R_substrates_acc (0.7238, 0.0347)
CNN_val_meta_guide_full_R_substrates_acc (0.7196, 0.0205)
CNN_val_merge_full_R_substrates_acc (0.7181, 0.0362)
CNN_val_multi_freeze_R_substrates_acc (0.7181, 0.0499)
CNN_val_meta_guide_freeze_R_substrates_acc (0.6823, 0.0356)

INH
inh R auc
C

In [9]:
# pd.DataFrame(df_total).to_csv('./303_performance.csv',index=False)